In [ ]:
import os
import sys
import shutil
import subprocess
from huggingface_hub import HfApi, login, snapshot_download

HF_TOKEN = os.getenv("HF_TOKEN")

# 변환할 원본 모델
SOURCE_MODEL_ID = "" 

# 업로드할 리포지토리
TARGET_REPO_ID = "" 

# 파일명 설정
GGUF_FILENAME_FP16 = "model-f16.gguf"
GGUF_FILENAME_Q4 = "model-Q4_K_M.gguf"

BASE_DIR = os.path.abspath(".")
MODEL_DIR = os.path.join(BASE_DIR, "downloaded_model")


LLAMA_CPP_DIR = os.path.join(BASE_DIR, "llama.cpp")

In [ ]:
def run_command(command, cwd=None):
    print(f"실행: {' '.join(command)}")
    try:
        subprocess.run(command, check=True, cwd=cwd)
    except subprocess.CalledProcessError as e:
        print(f"오류 발생: {e}")
        sys.exit(1)

In [ ]:
def load_model():
    if not HF_TOKEN:
        print("오류: HF_TOKEN 없음")
        sys.exit(1)
    login(token=HF_TOKEN)
    
    print(f"원본 모델: {SOURCE_MODEL_ID}")
    print(f"타겟 리포지토리: {TARGET_REPO_ID}")

    print(f"\n모델 다운로드: ({SOURCE_MODEL_ID})")
    if os.path.exists(MODEL_DIR):
        shutil.rmtree(MODEL_DIR)
        
    try:
        snapshot_download(
            repo_id=SOURCE_MODEL_ID,
            local_dir=MODEL_DIR,
            local_dir_use_symlinks=False,
            ignore_patterns=["*.gguf", "*.bin", "*.pth"]
        )
        print("✅ 다운로드 완료")
    except Exception as e:
        print(f"다운로드 실패: {e}")
        sys.exit(1)

In [ ]:
def main():
    if not os.path.exists(LLAMA_CPP_DIR):
        run_command(["git", "clone", "https://github.com/ggerganov/llama.cpp.git"])
    
    # 필수 패키지 설치
    run_command([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], cwd=LLAMA_CPP_DIR)
    
    # CMake 빌드 (양자화 도구 생성)
    print("llama.cpp 빌드 중...")
    build_dir = os.path.join(LLAMA_CPP_DIR, "build")
    run_command(["cmake", "-B", build_dir, "-DGGML_NATIVE=OFF"], cwd=LLAMA_CPP_DIR)
    run_command(["cmake", "--build", build_dir, "--config", "Release", "-j", "4"], cwd=LLAMA_CPP_DIR)
    
    # 양자화 실행 파일 위치 찾기
    quantize_bin = os.path.join(build_dir, "bin", "llama-quantize")
    if not os.path.exists(quantize_bin):
        # 경로가 다를 경우 (루트 빌드 등) 대비
        quantize_bin = os.path.join(build_dir, "llama-quantize")
        
    if not os.path.exists(quantize_bin):
        print("❌ 오류: llama-quantize 바이너리 빌드 실패.")
        sys.exit(1)

    print(f"\n[3/5] GGUF 변환 (FP16) 수행 중...")
    convert_script = os.path.join(LLAMA_CPP_DIR, "convert_hf_to_gguf.py")
    
    run_command([
        sys.executable, convert_script,
        MODEL_DIR,
        "--outfile", GGUF_FILENAME_FP16,
        "--outtype", "f16"
    ])
    
    if not os.path.exists(GGUF_FILENAME_FP16):
        print("❌ 변환 실패:")
        sys.exit(1)

    print(f"\n4bit 양자화")
    
    run_command([
        quantize_bin,
        GGUF_FILENAME_FP16,
        GGUF_FILENAME_Q4,
        "Q4_K_M"
    ])
    
    if not os.path.exists(GGUF_FILENAME_Q4):
        print("❌ 양자화 실패")
        sys.exit(1)
        
    print(f"✅ 양자화 완료: {GGUF_FILENAME_Q4}")
    
    # 용량 확보를 위해 FP16 파일 삭제
    os.remove(GGUF_FILENAME_FP16)
    shutil.rmtree(MODEL_DIR) 



if __name__ == "__main__":
    main()

In [ ]:
# 업로드
print(f"\n[5/5] Hugging Face 업로드 시작 ({TARGET_REPO_ID})...")
login(token=HF_TOKEN)    
api = HfApi()

api.create_repo(repo_id=TARGET_REPO_ID, exist_ok=True, repo_type="model")

try:
    api.upload_file(
        path_or_fileobj=GGUF_FILENAME_Q4,
        path_in_repo=GGUF_FILENAME_Q4,
        repo_id=TARGET_REPO_ID,
        repo_type="model"
    )
    print("\n업로드 완료")
    print(f"모델 링크: https://huggingface.co/{TARGET_REPO_ID}")
    
except Exception as e:
    print(f"❌ 업로드 중 오류 발생: {e}")


[5/5] Hugging Face 업로드 시작 (WindyAle/Kanana-nano-2.1B-Finance-v1-GGUF)...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


🎉 모든 작업이 완료되었습니다!
🔗 모델 링크: https://huggingface.co/WindyAle/Kanana-nano-2.1B-Finance-v1-GGUF
